<a href="https://colab.research.google.com/github/rikurzt/Finacial_analyze_course_HW/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_HW13_7113029027%E6%9D%8E%E5%86%A0%E5%BB%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 以PBR(股價淨值比)進行基因演算法

In [1]:
!pip install FinMind

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/151.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24860 sha256=f24bbb3f48f259f3e33a571e8b457a048f477a151dcc016a6f58cb3bce865ded
  Stored in directory: /root/.cache/pip/wheels/d4/bd/c4/0dda911b5461ee856352cd8c9e16472229bb51c95e74b74c08
Successfully built ta


In [6]:
from FinMind.data.finmind_api import pd
from FinMind.data import DataLoader

api = DataLoader()
# api.login_by_token(api_token='token')
# api.login(user_id='user_id',password='password')
stock_id = ['2330','4938','3706','2354','3231','3481']
pbr = []

for stock in stock_id:
  df = api.taiwan_stock_per_pbr(
      stock_id=stock,
      start_date='2024-01-01',
      end_date='2024-12-31',
  )
  avg_pbr = df['PBR'].mean()
  pbr.append(avg_pbr)

pbr

2024-12-06 08:30:25.678 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 2330
2024-12-06 08:30:26.915 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 4938
2024-12-06 08:30:27.588 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 3706
2024-12-06 08:30:28.359 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 2354
2024-12-06 08:30:29.036 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 3231
2024-12-06 08:30:29.806 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockPER, data_id: 3481


[6.234821428571429,
 1.367767857142857,
 0.9697767857142857,
 0.8614285714285713,
 2.8816964285714284,
 0.5847926267281106]

In [3]:
df

,date,stock_id,dividend_yield,PER,PBR
0,2024-01-02,4938,4.58,14.76,1.22
1,2024-01-03,4938,4.66,14.51,1.19
2,2024-01-04,4938,4.73,14.29,1.18
3,2024-01-05,4938,4.77,14.17,1.17
4,2024-01-08,4938,4.77,14.17,1.17
...,...,...,...,...,...
219,2024-11-29,4938,4.24,13.85,1.27
220,2024-12-02,4938,4.22,13.92,1.28
221,2024-12-03,4938,4.27,13.76,1.26
222,2024-12-04,4938,4.22,13.91,1.28


In [7]:
import numpy as np
import random


In [8]:
stocks = {
    'PBR': pbr
}

# 投資組合限制
MAX_STOCKS = 5  # 最大選股數量

# 參數設置
POPULATION_SIZE = 20  # 初始族群大小
GENERATIONS = 100     # 最大迭代次數
MUTATION_RATE = 0.1   # 突變概率
CROSSOVER_RATE = 0.8  # 交配概率

# 適應函數：目標是最小化加權平均 PBR
def fitness_function(chromosome):
    total_stocks = sum(chromosome)
    if total_stocks == 0 or total_stocks > MAX_STOCKS:  # 無效解或超過限制
        return float('inf')  # 適應度設為無效值（越大越差）
    total_pbr = sum(p * c for p, c in zip(stocks['PBR'], chromosome))
    return total_pbr / total_stocks  # 平均 PBR，越小越好

# 初始化族群
def initialize_population(size, n_genes):
    return [np.random.randint(0, 2, n_genes).tolist() for _ in range(size)]

# 選擇：輪盤選擇
def selection(population, fitnesses):
    total_fitness = sum(1 / f for f in fitnesses if f != float('inf'))  # 適應度反比
    probabilities = [(1 / f) / total_fitness if f != float('inf') else 0 for f in fitnesses]
    return population[np.random.choice(len(population), p=probabilities)]

# 交配：單點交叉
def crossover(parent1, parent2):
    if random.random() > CROSSOVER_RATE:
        return parent1, parent2
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# 突變
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return chromosome




In [13]:
def genetic_algorithm():
    n_genes = len(stocks['PBR'])
    population = initialize_population(POPULATION_SIZE, n_genes)

    for generation in range(GENERATIONS):
        fitnesses = [fitness_function(chromosome) for chromosome in population]
        next_generation = []

        while len(next_generation) < POPULATION_SIZE:
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            next_generation.extend([mutate(child1), mutate(child2)])

        population = next_generation[:POPULATION_SIZE]

        # 收集當前最佳解
        best_chromosome = min(population, key=fitness_function)
        best_fitness = fitness_function(best_chromosome)

        print(f"Generation {generation}: Best Fitness (Min PBR) = {best_fitness:.4f}")

    # 最終結果
    best_chromosome = min(population, key=fitness_function)
    selected_pbr = [p for p, c in zip(stocks['PBR'], best_chromosome) if c == 1]
    best_stock_id = [stock_id[i] for i, c in enumerate(best_chromosome) if c == 1]

    print("\nOptimal Portfolio:")
    print(f"Selection: {best_stock_id}")
    print(f"Selected PBR: {selected_pbr}")
    print(f"Average PBR: {fitness_function(best_chromosome):.4f}")

# 執行
genetic_algorithm()

Generation 0: Best Fitness (Min PBR) = 0.7773
Generation 1: Best Fitness (Min PBR) = 0.5848
Generation 2: Best Fitness (Min PBR) = 0.5848
Generation 3: Best Fitness (Min PBR) = 0.5848
Generation 4: Best Fitness (Min PBR) = 0.5848
Generation 5: Best Fitness (Min PBR) = 0.5848
Generation 6: Best Fitness (Min PBR) = 0.7231
Generation 7: Best Fitness (Min PBR) = 0.7231
Generation 8: Best Fitness (Min PBR) = 0.7773
Generation 9: Best Fitness (Min PBR) = 0.5848
Generation 10: Best Fitness (Min PBR) = 0.5848
Generation 11: Best Fitness (Min PBR) = 0.5848
Generation 12: Best Fitness (Min PBR) = 0.5848
Generation 13: Best Fitness (Min PBR) = 0.8614
Generation 14: Best Fitness (Min PBR) = 0.5848
Generation 15: Best Fitness (Min PBR) = 0.5848
Generation 16: Best Fitness (Min PBR) = 0.7773
Generation 17: Best Fitness (Min PBR) = 0.7231
Generation 18: Best Fitness (Min PBR) = 0.5848
Generation 19: Best Fitness (Min PBR) = 0.8053
Generation 20: Best Fitness (Min PBR) = 0.7231
Generation 21: Best Fit